# スケジュール算出プログラム

## ライブラリのインポート
pandasは表の処理

In [5]:
import random
import datetime
import numpy as np
import pandas as pd
from random import random
from array import array
import locale                               # ローカライズ用パッケージ

## 時間のローカライズ

In [6]:
locale.setlocale(locale.LC_TIME, "ja_JP")   # 日本にローカライズ（これがないと、英語の曜日が出えてくる）

'ja_JP'

## エクセルの読み込み関数

In [7]:
def read_excels():

    excel_path = "f_iden_3.xlsx"

    off_day_df  = pd.read_excel(excel_path, sheet_name="yukyu")                         # 休みの希望表
    skill_df    = pd.read_excel(excel_path, sheet_name="skill", header=[0, 1])          # スキル表
    due_date_df = pd.read_excel(excel_path, sheet_name="calendar")                      # 納期表

    # 空データ(NaN)の置換(0)
    off_day_df.fillna(0, inplace=True)
    # skill_df.fillna(0, inplace=True)
    due_date_df.fillna(0, inplace=True)

    # データの置換
    off_day_df.replace("◎", 2, inplace=True)                                            # 希望休（◎）を 2 に置換
    due_date_df.replace("☆", 3, inplace=True)                                          # 納期（☆）を 3 に置換

    # 列ごとのデータ型(dtype)を、intに変更しておく
    off_day_df[off_day_df.columns[2:]] = off_day_df[off_day_df.columns[2:]] .astype("int64")
    due_date_df[due_date_df.columns[3:]] = due_date_df[due_date_df.columns[3:]].astype("int64")

    # 品番の抽出
    product_num = due_date_df.loc[:, "品番"]

    # 土日の削除
    # データの中から土日の日付を取り出す
    off_days = [date for date in due_date_df.columns[3:] if date.strftime("%a") in "土日"]
    # 土日の日付を削除する
    off_day_df.drop(columns=off_days, inplace=True)
    due_date_df.drop(columns=off_days, inplace=True)

    return off_day_df, skill_df, due_date_df, product_num

In [8]:
off_day_df, skill_df, due_date_df, product_num = read_excels()

FileNotFoundError: [Errno 2] No such file or directory: 'f_iden_3.xlsx'

In [ ]:
off_day_df

,設備,備考,2023-06-01 00:00:00,2023-06-02 00:00:00,2023-06-05 00:00:00,2023-06-06 00:00:00,2023-06-07 00:00:00,2023-06-08 00:00:00,2023-06-09 00:00:00,2023-06-12 00:00:00,...,2023-06-19 00:00:00,2023-06-20 00:00:00,2023-06-21 00:00:00,2023-06-22 00:00:00,2023-06-23 00:00:00,2023-06-26 00:00:00,2023-06-27 00:00:00,2023-06-28 00:00:00,2023-06-29 00:00:00,2023-06-30 00:00:00
0,ヌートバー,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,近藤,0.0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
2,大谷,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,吉田,0.0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,2,0,0,0,0
4,岡本,0.0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
5,牧,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,源田,0.0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,甲斐,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,ダルビッシュ,0.0,0,0,0,0,0,0,0,0,...,0,0,0,2,0,0,0,0,0,0
9,高橋,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,2,0,0,0


In [ ]:
skill_df

スキルグループ                    
        1      2    3       4
0      近藤     大谷   吉田  ダルビッシュ
1      源田  ヌートバー   岡本      高橋
2       牧    NaN  NaN     NaN
3      甲斐    NaN  NaN     NaN

In [ ]:
due_date_df

,品番,スキルグループ,納期,2023-06-01 00:00:00,2023-06-02 00:00:00,2023-06-05 00:00:00,2023-06-06 00:00:00,2023-06-07 00:00:00,2023-06-08 00:00:00,2023-06-09 00:00:00,...,2023-06-19 00:00:00,2023-06-20 00:00:00,2023-06-21 00:00:00,2023-06-22 00:00:00,2023-06-23 00:00:00,2023-06-26 00:00:00,2023-06-27 00:00:00,2023-06-28 00:00:00,2023-06-29 00:00:00,2023-06-30 00:00:00
0,16204C49450,1,2023-06-12,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,16204C49450,2,2023-06-12,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,16204C49450,3,2023-06-12,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,16205G59900-1,1,2023-06-22,0,0,0,0,0,0,0,...,0,0,0,3,0,0,0,0,0,0
4,16205G59900-1,2,2023-06-22,0,0,0,0,0,0,0,...,0,0,0,3,0,0,0,0,0,0
5,16205G59900-1,3,2023-06-22,0,0,0,0,0,0,0,...,0,0,0,3,0,0,0,0,0,0
6,16205G59900-1,4,2023-06-22,0,0,0,0,0,0,0,...,0,0,0,3,0,0,0,0,0,0
7,1J07071011,1,2023-06-30,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
8,1J43171472B,1,2023-06-26,0,0,0,0,0,0,0,...,0,0,0,0,0,3,0,0,0,0
9,1J43171472B,2,2023-06-26,0,0,0,0,0,0,0,...,0,0,0,0,0,3,0,0,0,0


## 第1世代関数

In [ ]:
def first_gen(off_day_df: pd.DataFrame, skill_df: pd.DataFrame, due_date_df: pd.DataFrame, product_num: pd.Series):
    
    # 調整後の納期表（納期表のコピー）　→　以降、『調整納期表』　と呼ぶ
    adjusted_due_date_df = due_date_df.copy()


    # 作業者の割り振り　と　作業者の希望休を調整納期表にコピー
    worker_list = []
    # 行ごとに取り出して、作業者をランダムに割り振る
    for index, row in adjusted_due_date_df.iterrows():
       
        # 作業者を選ぶ
        workers: pd.DataFrame = skill_df.loc[:, (["スキルグループ"], [row["スキルグループ"]])]
        worker = workers.dropna().sample().values[0, 0]

        worker_list.append(worker)
        worker_off_days: pd.DataFrame = off_day_df.loc[off_day_df["設備"].isin([worker]), off_day_df.columns[2:]]                   # 作業者の希望休を抽出

        # 希望休を調整納期表にコピー（希望休ですべて上書きして納期を一度消す）
        adjusted_due_date_df.loc[[index], adjusted_due_date_df.columns[3:]] = worker_off_days.values

    # 割り当てたワーカーを adjusted_due_date_df へ「作業者」列として追加（※ここから adjusted_due_date_df が一列増えるので注意）
    adjusted_due_date_df.insert(2, "作業者", worker_list)


    # 作業者の希望休に合わせて、納期を調整
    after_index = None                                                                                                              # 後工程の index を保持する
    
    for index, row in adjusted_due_date_df[::-1].iterrows():                                                                        # due_date_df を逆順で行ごとに処理

        # 現工程の納期を抽出
        process: pd.Series = due_date_df.loc[index, due_date_df.columns[3:]]                                                        # 現工程の納期表データ
        process_due_date = process[process.isin([3])]                                                                               # 現工程の納期
        print("入力納期:", process_due_date.index[0])

        # 後工程の納期と本工程の納期を確認して調整
        # 後工程候補があるか（一つ目のデータは None なので無視）
        if after_index != None:

            print("[INFO]: 型番:", row["品番"], "=",adjusted_due_date_df.loc[after_index, "品番"])
            print("[INFO]: 工程:", row["スキルグループ"], "+ 1 =",adjusted_due_date_df.loc[after_index, "スキルグループ"])

            # 後工程があるか確認（後工程候補　と　型番が一致していて、かつ、スキルグループ（工程）が１つ次であるか）
            if row["品番"] == adjusted_due_date_df.loc[after_index, "品番"] and row["スキルグループ"] + 1 == adjusted_due_date_df.loc[after_index, "スキルグループ"]:

                print("後工程あり")

                # 後工程の納期を抽出
                after_process: pd.Series = adjusted_due_date_df.loc[after_index, adjusted_due_date_df.columns[4:]]                  # 後工程のデータ
                after_process_due_date = after_process[after_process.isin([3])]                                                     # 後工程の納期
                print("後工程の納期:", after_process_due_date.index[0])

                # 本工程の納期が後工程の納期より前に設定されていない場合　→　本工程の納期を後工程より前の日付にする
                if not process_due_date.index[0] < after_process_due_date.index[0]:

                    # 納期を１営業日早めて調整する（この時点では、有休をチェックしてないので決定しない）
                    before_days:pd.DataFrame = due_date_df.loc[[index], due_date_df.columns[3]:after_process_due_date.index[0]]     # 現在の納期より前のデータを抽出
                    new_due_date = before_days.columns[-2]                                                                          # 納期を１営業日早める
                    print("後工程を考慮した納期調整:", new_due_date)
                    
                    due_date_df.loc[index, process_due_date.index[0]] = 0                                                           # 現在の納期を納期表から削除
                    due_date_df.loc[index, new_due_date] = process_due_date.values[0]                                               # 新しい納期を納期表に追加

                    process: pd.Series = due_date_df.loc[index, due_date_df.columns[3:]]                                            # 現工程の納期表データ
                    process_due_date = process[process.isin([3])]                                                                   # 現工程の納期
                    print("現工程の納期:", process_due_date.index[0])

            else:

                print("後工程なし:")
        
        else:

            print("後工程なし（1つ目のデータ）:")
        
        
        # 希望休に合わせた調整
        # 現在の納期より前のデータを抽出
        before_days:pd.DataFrame = adjusted_due_date_df.loc[[index], adjusted_due_date_df.columns[4]:process_due_date.index[0]]

        # 納期を設定できるか確認していく
        for new_due_date in before_days.columns[::-1]:
                        
            # 納期を１日早める(-1日)
            print("納期候補:", new_due_date)

            # その日が希望休でない場合(0 のとき)　→　納期決定　→　ループ終了
            if adjusted_due_date_df.loc[index, new_due_date] == 0:

                # 納期が確定するのはココだけ
                print("納期決定:", new_due_date)
                adjusted_due_date_df.loc[index, new_due_date] = process_due_date.values[0]
                break


        after_index = index         # 後工程候補（一つ下の行）のindex

        print("-"*100)              # デバッグ用（ループごとに横線を表示）

    return adjusted_due_date_df


In [ ]:
#受取
adjusted_due_date_df = first_gen(off_day_df, skill_df, due_date_df, product_num)

入力納期: 2023-06-12 00:00:00
後工程なし（1つ目のデータ）:
納期候補: 2023-06-12 00:00:00
納期決定: 2023-06-12 00:00:00
----------------------------------------------------------------------------------------------------
入力納期: 2023-06-12 00:00:00
[INFO]: 型番: ZE1912020 = ZE1912020
[INFO]: 工程: 3 + 1 = 4
後工程あり
後工程の納期: 2023-06-12 00:00:00
後工程を考慮した納期調整: 2023-06-09 00:00:00
現工程の納期: 2023-06-09 00:00:00
納期候補: 2023-06-09 00:00:00
納期決定: 2023-06-09 00:00:00
----------------------------------------------------------------------------------------------------
入力納期: 2023-06-12 00:00:00
[INFO]: 型番: ZE1912020 = ZE1912020
[INFO]: 工程: 2 + 1 = 3
後工程あり
後工程の納期: 2023-06-09 00:00:00
後工程を考慮した納期調整: 2023-06-08 00:00:00
現工程の納期: 2023-06-08 00:00:00
納期候補: 2023-06-08 00:00:00
納期決定: 2023-06-08 00:00:00
----------------------------------------------------------------------------------------------------
入力納期: 2023-06-12 00:00:00
[INFO]: 型番: ZE1912020 = ZE1912020
[INFO]: 工程: 1 + 1 = 2
後工程あり
後工程の納期: 2023-06-08 00:00:00
後工程を考慮した納期調整: 2023-06-07 00

In [ ]:
off_day_df.iloc[:,:12]

,設備,備考,2023-06-01 00:00:00,2023-06-02 00:00:00,2023-06-05 00:00:00,2023-06-06 00:00:00,2023-06-07 00:00:00,2023-06-08 00:00:00,2023-06-09 00:00:00,2023-06-12 00:00:00,2023-06-13 00:00:00,2023-06-14 00:00:00
0,ヌートバー,0.0,0,0,0,0,0,0,0,0,0,0
1,近藤,0.0,0,0,0,0,0,0,0,2,2,0
2,大谷,0.0,0,0,0,0,0,0,0,0,0,0
3,吉田,0.0,0,0,0,0,0,0,0,0,0,0
4,岡本,0.0,0,0,0,0,0,0,2,0,0,0
5,牧,0.0,0,0,0,0,0,0,0,0,0,0
6,源田,0.0,0,0,0,2,0,0,0,0,0,0
7,甲斐,0.0,0,0,0,0,0,0,0,0,2,0
8,ダルビッシュ,0.0,0,0,0,0,0,0,0,0,0,0
9,高橋,0.0,0,0,0,0,0,0,0,0,0,0


In [ ]:
adjusted_due_date_df.iloc[-5:,:12]

,品番,スキルグループ,作業者,納期,2023-06-01 00:00:00,2023-06-02 00:00:00,2023-06-05 00:00:00,2023-06-06 00:00:00,2023-06-07 00:00:00,2023-06-08 00:00:00,2023-06-09 00:00:00,2023-06-12 00:00:00
47,ZE1325820A,3,吉田,2023-06-12,0,0,0,0,0,0,0,3
48,ZE1912020,1,牧,2023-06-12,0,0,0,0,3,0,0,0
49,ZE1912020,2,ヌートバー,2023-06-12,0,0,0,0,0,3,0,0
50,ZE1912020,3,吉田,2023-06-12,0,0,0,0,0,0,3,0
51,ZE1912020,4,ダルビッシュ,2023-06-12,0,0,0,0,0,0,0,3


In [ ]:
adjusted_due_date_df

,品番,スキルグループ,作業者,納期,2023-06-01 00:00:00,2023-06-02 00:00:00,2023-06-05 00:00:00,2023-06-06 00:00:00,2023-06-07 00:00:00,2023-06-08 00:00:00,...,2023-06-19 00:00:00,2023-06-20 00:00:00,2023-06-21 00:00:00,2023-06-22 00:00:00,2023-06-23 00:00:00,2023-06-26 00:00:00,2023-06-27 00:00:00,2023-06-28 00:00:00,2023-06-29 00:00:00,2023-06-30 00:00:00
0,16204C49450,1,牧,2023-06-12,0,0,0,0,0,3,...,0,0,0,0,0,0,0,0,0,0
1,16204C49450,2,大谷,2023-06-12,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,16204C49450,3,岡本,2023-06-12,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,16205G59900-1,1,牧,2023-06-22,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,16205G59900-1,2,大谷,2023-06-22,0,0,0,0,0,0,...,3,0,0,0,0,0,0,0,0,0
5,16205G59900-1,3,吉田,2023-06-22,0,0,0,0,0,0,...,2,3,0,0,0,2,0,0,0,0
6,16205G59900-1,4,ダルビッシュ,2023-06-22,0,0,0,0,0,0,...,0,0,3,2,0,0,0,0,0,0
7,1J07071011,1,甲斐,2023-06-30,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
8,1J43171472B,1,甲斐,2023-06-26,0,0,0,0,0,0,...,0,0,0,0,3,0,0,0,0,0
9,1J43171472B,2,大谷,2023-06-26,0,0,0,0,0,0,...,0,0,0,0,0,3,0,0,0,0


# 納期修正？

In [ ]:
#32
#休日数の修正
#工程数をカウントし、指定日～納期でランダムに選択。
hinban['工程番号']=0
hinban['品番数']=0
shuryo=[]
minlist=[]
maxlist=[]
nokilist=[]
def noki_fix(kiso_copy2, hinban, startup_day):
    kiso_copy2=kiso_copy2.iloc[:,3:]
  #工程カウント
    #hinban.loc[0,1]=0
    #hinban.loc[0,2]=0
    hinban.iloc[0,2]='min'
    for k2 in range(len(hinban)):
      if hinban.iloc[k2,0]== hinban.iloc[k2 +1,0]:
        hinban.iloc[k2 +1,1]+=1
        
      elif hinban.iloc[k2,0]!= hinban.iloc[k2 +1,0]: 
        hinban.iloc[k2+1 ,2]='max'
        hinban.iloc[k2 +2,2]='min'

    for k21 in range(len(hinban)):
      for d21 in range(len(kiso_copy2.columns)):
        if hinban.iloc[k21, 2]=='min' and kiso_copy2.iloc[k21,d21]==3:
          minlist.append(kiso_copy2.iloc[k21,d21].columns.values)

    for k22 in range(len(hinban)):
      for d22 in range(len(kiso_copy2.columns)):
        if hinban.iloc[k22, 2]=='max' and kiso_copy2.iloc[k22,d22]==3:
          maxlist.append(kiso_copy2.iloc[k22,d22].columns.values)


    nokilist.append(kiso2[1,2])
    for k23 in range(len(hinban)):
      #for d in range(len(kiso_copy2.columns)):
        if kiso2.iloc[k23,2]!=kiso2.iloc[k23 +1,2]:
          nokilist.append(kiso2.iloc[k23 +1,2])
          
  ##開始～終了間でk2個のランダム数作成→×品番毎にランダム数を小さい順に配置したい。
  #→n1,n2,..nk2を小さい順に並べ、工程間納期としたい。
 #削除予定
    list=[]
    if hinban.iloc[k2,2]!=0:
      
      while len(list)<k2:
        n=random.randint(startup_day,shuryo+1)
        if not n in list:
          list.append(n)
      newlist=list.sort()
      
      for k3 in range(len(hinban)):
        for d3 in range(len(kiso_copy2.columns)):
            if kiso_copy2.loc[k3,d3]==3:
              kiso_copy2.loc[k3,d3]=0
              for new in newlist:
                if kiso_copy2.iloc[k3,new]!=2:
                  kiso_copy2.iloc[k3,new]=3
                elif kiso_copy2.iloc[k3,new-1]!=2:
                  kiso_copy2.iloc[k3,new-1]=3
                elif kiso_copy2.iloc[k3,new-2]!=2:
                  kiso_copy2.iloc[k3,new-2]=3
                elif kiso_copy2.iloc[k3,new-3]!=2:
                  kiso_copy2.iloc[k3,new-3]=3
                elif kiso_copy2.iloc[k3,new-4]!=2:
                  kiso_copy2.iloc[k3,new-4]=3

  ##休日入力、 納期==3,休日==2→重なると休日優先で前工程の納期を全て１日前倒し→重なると休日優先で前工程の納期を全て１日前倒し
  #削除予定
  #  for j2 in range(len(hinban)):
  #      for ks2 in range(len(kiso2)):
  #        if kiso2.iat[len(hinban)-j2-1,0]== kiso2[len(kiso2)-ks2,1]:
  #          for m2 in range(days2):
  #            if kiso2.iat[len(kiso2) -ks2,days2 -m2 -1:]!=3:
  #              kiso2.iat[len(kiso2) -ks2 ,days2 -m2 -1:]=kiso2.iat[len(hinban)- j2 -1,days2 -m2-1:]
  #            elif kiso2.iat[len(kiso2) -ks2,days2 -m2 -1:]==3 and kiso2.iat[len(hinban) -j2 -1,days2 -m2-1:]==2:
  #              kiso2.iat[len(kiso2) -j2,days2 -m2 -1:]==2
  #              kiso2.iat[len(kiso2) -j2,days2 -m2 -2:]==3



    #if hinban.iat[k4 ,2]!=0:
        return kiso_copy2, hinban, minlist, maxlist, nokilist

NameError: name 'hinban' is not defined

In [ ]:
#kiso_copy2= noki_fix()
kiso_copy2, product_num, minlist, maxlist, nokilist=noki_fix(kiso_copy2, hinban, startup_day)

NameError: name 'noki_fix' is not defined

In [ ]:
#42
##(1)工程ができるだけ長くならないこと
##(2)前倒し生産方式の場合、できるだけ開始日に近いこと、ジャストインタイム方式の場合、できるだけ納期に近いこと。
#評価方法
koteikan=0
hoshiki=0
if kiso2.iloc[0,6]=='〇':
  h=1
else:
  h=0
#for k3 in range(len(hinban)):
#        for d3 in range(len(kiso_copy2.columns)):
def evalution_function2(startup_day, minlist, maxlist):
##(1)工程ができるだけ長くならないこと
#  for d3 in range(len(kiso_copy2)):
#    for e3 in range(len(hinban)):
#      kouteikan=((d3(max)-d3(min))^2)^0.5
  #for k4 in range(len(hinban)):
  #  for d4 in 2:
  #    if hinban(k4,0)== hinban(k4 +1,0):
  #      kouteikan=((d4(max)-d4(min))^2)^0.5
  for k4 in maxlist:
    koteikan=-1*sum(abs(maxlist[k4]- minlist[k4]))
    
##(2)前倒し生産方式の場合、できるだけ開始日に近いこと、ジャストインタイム方式の場合、できるだけ納期に近いこと。
  #①前倒し時スタート-第１工程近い
  #startup_day
 # houshiki=0
  if h== 0:
   #for d4 in range(len(kiso_copy2)):
   # for e4 in range(len(hinban)):
   #   if hinban.iloc[e4,1]==1:
   #     kiso_copy2.iloc[e4,d4]
   #       houshiki=-1*((d4(start)-d4(min))^2)^0.5
    for k41 in maxlist:
      hoshiki=-1*sum(abs(minlist[k41])-startup_day)
      
  #②ジャストインタイム時納期-最終工程近い
  hoshiki=0
  if h== 1:
   #for d4 in range(len(kiso_copy2)):
   # for e4 in range(len(hinban)):
   #   kouteikan=-1*((d4(noki)-d4(max))^2)^0.5
    for k42 in maxlist:
      hoshiki=-1*sum(abs(nokilist[k42])-maxlist[k42])



    #評価
    score2+=koteikan+hoshiki
    return score2
    #print(score)

In [ ]:
##52関数化
#1エクセル読み込み
#kiso2,holiday=read_excel()
due_date_df,product_num=read_excels()
#2第1世代
#kiso_copy=first_gene(kiso,holiday)
kiso_copy2=first_gen(kiso_copy2, days2)
#3休日数の修正
#kiso_copy=holiday_fix(kiso_copy,holiday)
kiso_copy2=noki_fix(kiso_copy2, product_num, minlist, maxlist, nokilist)
#4評価
#score=evalution_function(kiso_copy)
score2=evalution_function2(koteikan, hoshiki)
score2

In [ ]:
##62遺伝的アルゴリズム

#def read_excel():
    #Excel読み込み
#    df=pd.read_excel('iden.xlsm',sheet_name='Sheet1')
#一様交叉,ep=0.5,sd=0.05
def crossover2(ep2,sd2,p21,p22):
    #1か月の日数
    days2=len(p21.columns)
    #1次元化
    p21=np.array(p21).flatten()
    p22=np.array(p22).flatten()
    #子の変数
    ch21=[]
    ch22=[]
    for p21_,p22_ in zip(p21,p22):
        x = True if ep2 >random() else False
        if x==True:
            ch21.append(p21_)
            ch22.append(p22_)
        else:
            ch21.append(p22_)
            ch22.append(p21_)
    #突然変異
    ch21,ch22=mutation2(sd2,np.array(ch21).flatten(),np.array(ch22).flatten())
    #pandasに変換
    ch21=pd.DataFrame(ch21.reshape(int(len(ch21)/days2),days2))
    ch22=pd.DataFrame(ch22.reshape(int(len(ch22)/days2),days2))
    #列名の変更
    ch21.columns=[i+1 for i in range(len(ch21.columns))]
    ch22.columns=[i+1 for i in range(len(ch22.columns))]
    return ch21,ch22
#突然変異
def mutation2(sd2,ch21,ch22):
    x = True if sd2>random() else False
    if x==True:
        rand=np.random.permutation([i for i in range(len(ch21))])
        #遺伝子の10％を変異させる
        #並び換えた先頭10％
        rand=rand[:int(len(ch21)//10)]
        for i in rand:
            #1なら0,0なら1
            if ch21[i]==1:
                ch21[i]==0

            if ch21[i]==0:
                ch21[i]==1
                #ch2も同じことをする。
    x=True if sd2>random() else False
    if x==True:
        rand2=np.random.permutation([i for i in range(len(ch21))])
        #並び換えた先頭10％
        rand2=rand[:int(len(ch21)//10)]
        for i in rand2:
            #1なら0,0なら1
            if ch22[i]==1:
                ch22[i]==0
            if ch22[i]==0:
                ch22[i]==1

    return ch21,ch22


In [ ]:
##72全体
#遺伝的アルゴリズム
#エクセル読み込み
off_day_df,product_num=read_excels()
#親の保存
parent2=[]
for i in range(100):
    #第1世代
    kiso_copy2=first_gen(due_date_df, product_num)
    #休日数の修正
    kiso_copy2=noki_fix(kiso_copy2, product_num, startup_day)
    #parent.append(kiso_copy)
    #評価
    score2=evalution_function2(kiso_copy2)
    #第1世代を格納
    parent2.append([score2, kiso_copy2])
#上位個体数
elite_length2=20
#世代数
gene_length2=50

#一様交叉確立
ep2=0.5
#突然変異確立
sd2=0.05

#一様交叉
#ch1 = pd.DataFrame(ch1)
#ch2 = pd.DataFrame(ch2)
#ch1,ch2=crossover(ep,sd,parent[0],parent[1])
#休日数変更
#ch1=holiday_fix(ch1,holiday)
#ch2=holiday_fix(ch2,holiday)

for i in range(gene_length2):
    #点数で並び換え
    parent2 = sorted(parent2, key=lambda x: -x[0])
    #parent=sorted(np.array(parent), key=lambda x:-x[0])
    #parent=sorted(np.array(parent[0]).flatten(), key=lambda x:-x[0])
    #p1=np.array(parent[0]).flatten()
    #上位個体を選別
    parent2=parent2[:elite_length2]
    #各世代
    print('第'+str(i+1)+'世代')

    #最高得点の更新
    #if i ==0 or top[0]<parent[0][0]:
    if i ==0:
        top2=parent2[0]
            #各世代の最高得点の表示
        print(top2[0])
        print(np.array(top2[1]))
    else:
        if top2[0]>parent2[0][0]:
            parent2.append(top2)
            #各世代の最高得点の表示
            print(top2[0])
            print(np.array(top2[1]))
        else:
            top2=parent2[0]
            #各世代の最高得点の表示
            print(parent2[0][0])
            print(np.array(parent2[0][1]))

    #各世代
    #print('第'+str(i+1)+'世代')
    #各世代の最高得点の表示
    #print(top[0])
    #print(np.array(top[1]))

    #子世代
    children2=[]
    #遺伝子操作
    for k21,v21 in enumerate(parent2):
        for k22,v22 in enumerate(parent2):
            if k21<k22:
                #一様交叉
                ch21,ch22=crossover2(ep2,sd2,v21[1],v22[1])
                #休日数変更
                ch21=noki_fix(ch21,product_num)
                ch22=noki_fix(ch22,product_num)
                #評価
                score21=evalution_function2(ch21)
                score22=evalution_function2(ch22)
                #子孫を格納
                children2.append([score21,ch21])
                children2.append([score22,ch22])

    #子を親にコピー
    parent2=children2.copy()

#最強個体の保存
x=top2[1].replace(3,'〇').replace(2,'◎').replace(0,'')
x.to_excel('f_shiftf_32.xlsx')


In [ ]:
##72全体
#遺伝的アルゴリズム
#エクセル読み込み
off_day_df,product_num=read_excels()
#親の保存
parent2=[]
for i in range(100):
    #第1世代
    kiso_copy2=first_gen(due_date_df, product_num)
    #休日数の修正
    kiso_copy2=noki_fix(kiso_copy2, product_num, startup_day)
    #parent.append(kiso_copy)
    #評価
    score2=evalution_function2(kiso_copy2)
    #第1世代を格納
    parent2.append([score2, kiso_copy2])
#上位個体数
elite_length2=20
#世代数
gene_length2=50

#一様交叉確立
ep2=0.5
#突然変異確立
sd2=0.05

#一様交叉
#ch1 = pd.DataFrame(ch1)
#ch2 = pd.DataFrame(ch2)
#ch1,ch2=crossover(ep,sd,parent[0],parent[1])
#休日数変更
#ch1=holiday_fix(ch1,holiday)
#ch2=holiday_fix(ch2,holiday)

for i in range(gene_length2):
    #点数で並び換え
    parent2 = sorted(parent2, key=lambda x: -x[0])
    #parent=sorted(np.array(parent), key=lambda x:-x[0])
    #parent=sorted(np.array(parent[0]).flatten(), key=lambda x:-x[0])
    #p1=np.array(parent[0]).flatten()
    #上位個体を選別
    parent2=parent2[:elite_length2]
    #各世代
    print('第'+str(i+1)+'世代')

    #最高得点の更新
    #if i ==0 or top[0]<parent[0][0]:
    if i ==0:
        top2=parent2[0]
            #各世代の最高得点の表示
        print(top2[0])
        print(np.array(top2[1]))
    else:
        if top2[0]>parent2[0][0]:
            parent2.append(top2)
            #各世代の最高得点の表示
            print(top2[0])
            print(np.array(top2[1]))
        else:
            top2=parent2[0]
            #各世代の最高得点の表示
            print(parent2[0][0])
            print(np.array(parent2[0][1]))

    #各世代
    #print('第'+str(i+1)+'世代')
    #各世代の最高得点の表示
    #print(top[0])
    #print(np.array(top[1]))

    #子世代
    children2=[]
    #遺伝子操作
    for k21,v21 in enumerate(parent2):
        for k22,v22 in enumerate(parent2):
            if k21<k22:
                #一様交叉
                ch21,ch22=crossover2(ep2,sd2,v21[1],v22[1])
                #休日数変更
                ch21=noki_fix(ch21,product_num)
                ch22=noki_fix(ch22,product_num)
                #評価
                score21=evalution_function2(ch21)
                score22=evalution_function2(ch22)
                #子孫を格納
                children2.append([score21,ch21])
                children2.append([score22,ch22])

    #子を親にコピー
    parent2=children2.copy()

#最強個体の保存
x=top2[1].replace(3,'〇').replace(2,'◎').replace(0,'')
x.to_excel('f_shiftf_32.xlsx')
